In [15]:
from itertools import chain, combinations, product

def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

def is_vi_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            if not(x in v):
                return False
    return True

def is_vii_satisfied(collection_of_neighborhood_systems, powset_minus_empty_set_and_S, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            for v_ in powset_minus_empty_set_and_S:
                if set(v) <= set(v_):
                    if not(v_ in collection_of_neighborhood_systems[x]):
                        return False
    return True

def is_viii_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v1, v2 in combinations(collection_of_neighborhood_systems[x], 2):
            if not(tuple(sorted(set(v1) & set(v2))) in collection_of_neighborhood_systems[x]):
                return False
    return True

def is_viv_satisfied(collection_of_neighborhood_systems, n):
    for x in range(1, n + 1):
        for v in collection_of_neighborhood_systems[x]:
            for w in collection_of_neighborhood_systems[x]:
                flag = True
                for y in w:
                    if not(v in collection_of_neighborhood_systems[y]):
                        flag = False
                        break
                if flag:
                    break
            if not(flag):
                return False
    return True

def all_open_sets(list_of_collections_of_neighborhood_systems, powset_of_S):
    list_of_sets_of_open_sets = []
    for collection_of_neighborhood_systems in list_of_collections_of_neighborhood_systems:
        set_of_open_sets = []
        for o in powset_of_S:
            flag = True
            for x in o:
                if not(o in collection_of_neighborhood_systems[x]):
                    flag = False
            if flag:
                set_of_open_sets.append(o)
        list_of_sets_of_open_sets.append(tuple(set_of_open_sets))
    return list_of_sets_of_open_sets

def all_closed_sets(list_of_sets_of_open_sets, S):
    return [tuple(tuple(sorted(set(S) - set(s))) for s in set_of_open_sets) for set_of_open_sets in list_of_sets_of_open_sets]

def all_open_kernel_operators(list_of_collections_of_neighborhood_systems, powset_of_S):
    list_of_open_kernel_operators = []
    for collection_of_neighborhood_systems in list_of_collections_of_neighborhood_systems:
        open_kernel_operator = {}
        for v in powset_of_S:
            vo = []
            for x in range(1, len(collection_of_neighborhood_systems) + 1):
                if v in collection_of_neighborhood_systems[x]:
                    vo.append(x)
            open_kernel_operator[v] = tuple(vo)
        list_of_open_kernel_operators.append(open_kernel_operator)
    return list_of_open_kernel_operators

def all_closure_operators(list_of_sets_of_closed_sets, powset_of_S):
    list_of_closure_operators = []
    for set_of_closed_sets in list_of_sets_of_closed_sets:
        closure_operators = {}
        for s in powset_of_S:
            for cs in sorted(set_of_closed_sets, key = lambda s: len(s)):
                if set(s) <= set(cs):
                    closure_operators[s] = cs
                    break
        list_of_closure_operators.append(closure_operators)
    return list_of_closure_operators

def all_collections_of_neighborhood_systems(n, powset_of_S, S):
    powset_minus_empty_set_and_S = tuple(s for s in powset_of_S if s != () and s != S)
    list_of_collections_of_neighborhood_systems = []

    for t in product(powerset(powset_minus_empty_set_and_S), repeat = n):
        collection_of_neighborhood_systems = {}
        for i in range(1, n + 1):
            collection_of_neighborhood_systems[i] = t[i - 1]
        if not(is_vi_satisfied(collection_of_neighborhood_systems, n)):
            continue
        if not(is_vii_satisfied(collection_of_neighborhood_systems, powset_minus_empty_set_and_S, n)):
            continue
        if not(is_viii_satisfied(collection_of_neighborhood_systems, n)):
            continue
        if not(is_viv_satisfied(collection_of_neighborhood_systems, n)):
            continue
        collection_of_neighborhood_systems = {x:(collection_of_neighborhood_systems[x] + (S,)) for x in collection_of_neighborhood_systems.keys()}
        list_of_collections_of_neighborhood_systems.append(collection_of_neighborhood_systems)
    return list_of_collections_of_neighborhood_systems

def all_topologies(n):
    """
    S = {1, 2, ..., n}
    """
    powset_of_S = tuple(powerset(range(1, n + 1)))
    powset_of_S = tuple(tuple(sorted(t)) for t in powset_of_S)
    S = tuple(range(1, n + 1))
    list_of_collections_of_neighborhood_systems = all_collections_of_neighborhood_systems(n, powset_of_S, S)
    list_of_open_kernel_operators = all_open_kernel_operators(list_of_collections_of_neighborhood_systems, powset_of_S)
    list_of_sets_of_open_sets = all_open_sets(list_of_collections_of_neighborhood_systems, powset_of_S)
    list_of_sets_of_closed_sets = all_closed_sets(list_of_sets_of_open_sets, S)
    list_of_closure_operators = all_closure_operators(list_of_sets_of_closed_sets, powset_of_S)
    return (list_of_sets_of_open_sets, list_of_sets_of_closed_sets, list_of_open_kernel_operators, list_of_closure_operators, list_of_collections_of_neighborhood_systems)

In [16]:
import pickle
t = all_topologies(3)
with open('topology_from_collections_of_neighborhood_systems.dump', 'wb') as f:
    pickle.dump(t, f)

(l1, l2, l3, l4, l5) = t
print("All Topologies of {1, 2, 3}:")
print()
for i in range(len(l1)):
    if i == 0:
        print(str(i+1) + "st Topology:")
    if i == 1:
        print(str(i+1) + "nd Topology:")
    if i == 2:
        print(str(i+1) + "rd Topology:")
    if i >= 3:
        print(str(i+1) + "th Topology:")
    print("Open Sets: ")
    print(l1[i])
    print("Closed Sets: ")
    print(l2[i])
    print("Open Kernel Operator: ")
    print(l3[i])
    print("Closure Operator: ")
    print(l4[i])
    print("Neighborhood Systems: ")
    print(l5[i])
    print()

All Topologies of {1, 2, 3}:

1st Topology:
Open Sets: 
((), (1, 2, 3))
Closed Sets: 
((1, 2, 3), ())
Open Kernel Operator: 
{(): (), (1,): (), (2,): (), (3,): (), (1, 2): (), (1, 3): (), (2, 3): (), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2, 3), (2,): (1, 2, 3), (3,): (1, 2, 3), (1, 2): (1, 2, 3), (1, 3): (1, 2, 3), (2, 3): (1, 2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1, 2, 3),), 2: ((1, 2, 3),), 3: ((1, 2, 3),)}

2nd Topology:
Open Sets: 
((), (3,), (1, 2, 3))
Closed Sets: 
((1, 2, 3), (1, 2), ())
Open Kernel Operator: 
{(): (), (1,): (), (2,): (), (3,): (3,), (1, 2): (), (1, 3): (3,), (2, 3): (3,), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2), (2,): (1, 2), (3,): (1, 2, 3), (1, 2): (1, 2), (1, 3): (1, 2, 3), (2, 3): (1, 2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1, 2, 3),), 2: ((1, 2, 3),), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

3rd Topology:
Open Sets: 
((), (2, 3), (1, 2, 3))
Closed Sets: 
((1, 2, 3), (1,), ())


23th Topology:
Open Sets: 
((), (1,), (3,), (1, 2), (1, 3), (1, 2, 3))
Closed Sets: 
((1, 2, 3), (2, 3), (1, 2), (3,), (2,), ())
Open Kernel Operator: 
{(): (), (1,): (1,), (2,): (), (3,): (3,), (1, 2): (1, 2), (1, 3): (1, 3), (2, 3): (3,), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2), (2,): (2,), (3,): (3,), (1, 2): (1, 2), (1, 3): (1, 2, 3), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1,), (1, 2), (1, 3), (1, 2, 3)), 2: ((1, 2), (1, 2, 3)), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

24th Topology:
Open Sets: 
((), (1,), (2, 3), (1, 2, 3))
Closed Sets: 
((1, 2, 3), (2, 3), (1,), ())
Open Kernel Operator: 
{(): (), (1,): (1,), (2,): (), (3,): (), (1, 2): (1,), (1, 3): (1,), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1,), (2,): (2, 3), (3,): (2, 3), (1, 2): (1, 2, 3), (1, 3): (1, 2, 3), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1,), (1, 2), (1, 3), (1, 2, 3)), 2: ((2, 3), (1, 2, 3)), 3: ((2, 3), 